# Continuous Time Sphere Model

We model the example of short-lived temperature sensors on the Earth in continuous time, using a combination of a queueing model and point process. Throughout the setup of the model, fix a time $T;$ for each time $t \in [0,T]$ the set of sensors $S_t$ at time $t$ is a set of locations on $\mathbb{S}^2,$ where $|S_t|$ is a random variable, and $S_t(i),$ the location of the $i$th sensor is uniformly sampled from the uniform measure on the sphere.

Perhaps the easiest way to set up the model is via a (FIFO) queueing model, or birth-death process representing the sensors:

- We start with $L_0$ number of sensors, each sampled uniformly from the sphere. 
- Sensors are added to the queue every so often, again sampled uniformly from the sphere. 
- Independently of the creation of new sensors, sensors break down at a perhaps different rate, and leave the queue.

We can model the time between consecutive joining/ exiting of the queue by exponential distributions.


## Queueing Model Implementation
Consider a queueing model whose initial length is $L_0,$ the number of initial sensors we begin with. We assume that the queue processes sensors (physically: sensors break down) in a random time $\sim \text{Exp}(\lambda_1),$ and sensors join the queue with interarrival times distributed according to $\text{Exp}(\lambda_2),$ which are independent of the times it takes for the queue to process the input.

We can simulate this queue in the following way. For a given fixed time $t \in [0,T],$ by the memoryless property of exponential distributions, the time until a sensor leaves the queue is still $X_1 \sim \text{Exp}(\lambda_1)$ and the interarrival time is still distributed according to $X_2 \text{Exp}(\lambda_2).$ Now, the time until one of these two actions occurs is a random variable $Z = \min(X_1,X_2) \sim \text{Exp}(\lambda_1+\lambda_2)$ (which can be checked). Now the probability that a sensor joins the queue first is $\mathbb{P}(X_1 > X_2) = \frac{\lambda_1}{\lambda_1 + \lambda_2},$ and one minus that for the probability that a sensor breaks and leaves the queue. We can then model the process by waiting $\text{Exp}(\lambda_1+\lambda_2)$ amount of time, and with probability $\frac{\lambda_1}{\lambda_1 + \lambda_2},$ adding a new sensor to the queue, or otherwise removing the oldest sensor from the queue.

## Dynamic Network Analysis
The continuous time model allows us to subsample as finely as we want. We keep track of the birth/death of the sensors in an interval tree; for any $t \in [0,T]$ we query the interval tree for the set of points that are present on the sphere at that time, and then create a dynamic network via the delaunay construction, assigining node values via some observation function representing temperature, or another desired quantity, and the edge weights by distance on the sphere.

In [1]:
def get_sensor_lifetimes(time, birth_rate, death_rate):
    '''
    A shoddy implementation of a queueing model, with exponentially distributed processing time and
    exponentially distributed growth.
    '''
    l1 = 1/birth_rate
    l2 = 1/death_rate

    births = [0]*100 #initialize 100 points

    current_time = 0
    intervals = it.IntervalTree()

    while current_time < time:

        # if there are no current points
        if len(births) == 0:
            current_time += np.random.exponential(l2)
            births.append(current_time)
            continue

        current_time += np.random.exponential(l1 + l2)

        # add a new point
        if np.random.rand() < l1/(l1 + l2):
            births.append(current_time)
        else:
            intervals[births.pop(0):current_time] = sample_uniform_sphere(1)

    return intervals


def create_dynamic_networks(lambda1,lambda2, obs_times, observation_function, edge_wtsfn):
    '''
    Can query the interval tree below for more information about the points
    -----------------------------------------------------------------------
    Input:
    lambda1: birth rate (we expect )
    lambda2: death rate (length of a lifetime of a node)
    obs_times - list of times at which to sample obs function
    observation_function - function from which to sample
    edge_wtsfn - function applied to edge wts
    '''

    ### create the interval tree ###
    intervals = get_sensor_lifetimes(max(obs_times), lambda1, lambda2)

    ### query tree at each timestep ###
    coordinate_set = []
    for t in obs_times:
        points = intervals.at(t)
        coordinates = [ p[2] for p in list(points) ]

        coordinate_set.append(coordinates)

        hull = sp.ConvexHull(coordinates)
        node_wts.append(get_node_wts(t,hull,observation_function))
        edge_wts.append(edge_wtsfn(hull))

    return (node_wts,edge_wts,point_set)